In [ ]:
!pip install tensorflow
!pip install tensorflow-privacy

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_privacy as tfp

from matplotlib import pyplot as plt
from IPython.display import display, HTML, IFrame

In [ ]:
# Load the training and test datasets from the MNIST dataset
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

In [ ]:
# Verify the shape of the training data
# 60k training 28X28 images
train_data.shape

In [ ]:
train_labels.shape

In [ ]:
# Plot some training examples from the MNIST dataset
fig = plt.figure(figsize=(20,4))
j = 0

for i in range(40):
  plt.subplot(4, 10, j + 1)
  plt.imshow(train_data[i], cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

In [ ]:
# Preprocess the MNIST data for training
train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 784)
test_data = test_data.reshape(test_data.shape[0], 784)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

assert train_data.min() >= 0.
assert train_data.max() <= 1.
assert test_data.min() >= 0.
assert test_data.max() <= 1.

In [ ]:
# Define the Hyperparameters of our DP model
LR = 0.3
EPOCHS = 100
BATCH_SIZE = 250
L2_NORM_CLIP = 20
NOISE_MULTIPLIER = 0.1
NUM_MICROBATCHES = 1

if BATCH_SIZE % NUM_MICROBATCHES != 0:
  raise ValueError('Batch size shoyld be multiple of the number of microbatches')

In [ ]:
# Construct the Keras NN used for classification
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(784,)),
    tf.keras.layers.Dense(10, kernel_initializer='zeros'),
    tf.keras.layers.Softmax()
])

In [ ]:
# Define the DP-SGD optimizer and the loss function
optimizer = tfp.DPKerasSGDOptimizer(
    l2_norm_clip=L2_NORM_CLIP,
    noise_multiplier=NOISE_MULTIPLIER,
    num_microbatches=NUM_MICROBATCHES,
    learning_rate=LR
)

loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [ ]:
# Compile with the optimizer, loss and metrics and train the model in DP fashion
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

history = model.fit(
    x=train_data,
    y=train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

In [ ]:
# Evaluate the DP-SGD model
model.evaluate(x=test_data, y=test_labels)

In [ ]:
# Compute the differential privacy guarantees of our model
print(tfp.privacy.analysis.compute_dp_sgd_privacy_lib.compute_dp_sgd_privacy_statement(
  number_of_examples=train_data.shape[0],
  batch_size=BATCH_SIZE,
  num_epochs=EPOCHS,
  noise_multiplier=NOISE_MULTIPLIER,
  delta=1e-5
))